In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

train = pd.read_csv("./train_cleaned.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

y = train[list_classes].values

#Stratify KFold by creating unique labels for ground truth values, toss things with less than num_folds into an 'others' label
num_folds = 10
truth_labels = np.unique(y,axis=0)

truth_counts = np.zeros(truth_labels.shape[0],dtype=int)
for j, tlab in enumerate(truth_labels):
    truth_counts[j] = (y == tlab).all(-1).sum()

other_label = truth_labels.shape[0]+1 #create a new label out of the index range for 'others'
truth_encoded = []
for row in y:
    for j, truth_row in enumerate(truth_labels):
        index_test = np.array_equal(row, truth_row)
        if index_test:
            if truth_counts[j]>=num_folds:
                truth_encoded.append(j)
            else:
                truth_encoded.append(other_label)

                
kf = StratifiedKFold(n_splits=num_folds, random_state=42, shuffle=True)
temp = pd.read_csv("./train_cleaned.csv")
temp.drop(['comment_text']+ list_classes,axis=1, inplace=True)
temp['split'] = np.zeros(temp.shape[0],dtype=np.int32)
for j, (train_index, val_index) in enumerate(kf.split(np.zeros(len(truth_encoded)), truth_encoded)):
    temp['split'][val_index]=j
temp.to_csv('splits.csv', index=False)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
